In [1]:
import pandas as pd
import numpy as np

# Step 1: Create the initial dataset with dates starting from January 1, 2024
dates = pd.date_range(start='2024-01-01', periods=60)
df = pd.DataFrame({'Date': dates})

# Initialize the fruit prices on the first day
initial_price = 1.00
df['Apple_Price'] = initial_price
df['Orange_Price'] = initial_price
df['Banana_Price'] = initial_price

# Step 2 & 3: For each day after day 1, update the prices for each fruit
for i in range(1, len(df)):
    # Generating random price changes for each fruit
    apple_change = np.random.normal(loc=0.05, scale=0.01)
    orange_change = np.random.normal(loc=0.05, scale=0.01)
    banana_change = np.random.normal(loc=0.05, scale=0.01)
    
    # Update the prices based on the previous day's price and the random change
    df.loc[i, 'Apple_Price'] = df.loc[i-1, 'Apple_Price'] + apple_change
    df.loc[i, 'Orange_Price'] = df.loc[i-1, 'Orange_Price'] + orange_change
    df.loc[i, 'Banana_Price'] = df.loc[i-1, 'Banana_Price'] + banana_change

# Ensuring no negative prices
df[['Apple_Price', 'Orange_Price', 'Banana_Price']] = df[['Apple_Price', 'Orange_Price', 'Banana_Price']].clip(lower=0)

print(df.head())  # Print the first few rows to check

        Date  Apple_Price  Orange_Price  Banana_Price
0 2024-01-01     1.000000      1.000000      1.000000
1 2024-01-02     1.055473      1.036774      1.072811
2 2024-01-03     1.115016      1.078999      1.134943
3 2024-01-04     1.159149      1.132265      1.180590
4 2024-01-05     1.198899      1.181403      1.235064


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Assuming `df` is your DataFrame from the previous code snippet with the price data for apples, oranges, and bananas.

# Step 1: Calculate daily returns for each fruit
returns = df[['Apple_Price', 'Orange_Price', 'Banana_Price']].pct_change().dropna()

# Step 2: Calculate mean return and standard deviation of daily returns
mean_returns = returns.mean()
std_devs = returns.std()

# Simplified Sharpe Ratio calculation (without risk-free rate)
def sharpe_ratio(weights, mean_returns, std_devs):
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    return -portfolio_return / portfolio_std  # Negative because we want to maximize this in minimization function

# Constraint: Sum of weights = 1 (100% of the portfolio)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Boundaries: Each weight can be between 0 and 1 (can't hold a negative amount of fruit)
bounds = tuple((0, 1) for _ in range(returns.shape[1]))

# Initial guess (even distribution)
init_guess = [1./returns.shape[1] for _ in range(returns.shape[1])]

# Optimization
optimal = minimize(sharpe_ratio, init_guess, args=(mean_returns, std_devs), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal allocation
optimal_allocation = optimal.x

print("Optimal allocation:")
print(f"Apples: {optimal_allocation[0]*100:.2f}%")
print(f"Oranges: {optimal_allocation[1]*100:.2f}%")
print(f"Bananas: {optimal_allocation[2]*100:.2f}%")

Optimal allocation:
Apples: 26.44%
Oranges: 73.56%
Bananas: 0.00%


In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Create the initial dataset with dates starting from January 1, 2024
dates = pd.date_range(start='2024-01-01', periods=100)  # Adjusted to 100 days
df = pd.DataFrame({'Date': dates})

# Initialize the fruit prices on the first day
initial_price = 1.00
df['Apple_Price'] = initial_price
df['Orange_Price'] = initial_price
df['Banana_Price'] = initial_price

# Initialize a DataFrame to hold the optimal allocation for each day
allocation_df = pd.DataFrame(index=df.index, columns=['Apple_Alloc', 'Orange_Alloc', 'Banana_Alloc'])
allocation_df.iloc[0] = [1/3, 1/3, 1/3]  # Start with an equal allocation

# Function to calculate the negative Sharpe Ratio
def neg_sharpe_ratio(weights, returns):
    portfolio_return = np.dot(weights, returns.mean())
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
    return -portfolio_return / portfolio_std

# Constraint: Sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Bounds for the weights
bounds = ((0, 1), (0, 1), (0, 1))

# Loop through each day, calculate prices
for i in range(1, len(df)):
    # Update prices
    for fruit in ['Apple', 'Orange', 'Banana']:
        price_change = np.random.normal(loc=0.05, scale=0.01)
        df.loc[i, f'{fruit}_Price'] = df.loc[i-1, f'{fruit}_Price'] + price_change

# Calculate daily returns
returns = df[['Apple_Price', 'Orange_Price', 'Banana_Price']].pct_change().dropna()

# Optimization only for the last 7 days
for i in range(len(df) - 7, len(df)):
    # Optimization to find new allocation
    initial_guess = [1/3, 1/3, 1/3]  # Start with equal allocation for simplicity
    opt_result = minimize(neg_sharpe_ratio, initial_guess, args=(returns[:i],), method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Store the new optimal allocation
    allocation_df.iloc[i] = opt_result.x

# Ensuring no negative prices
df[['Apple_Price', 'Orange_Price', 'Banana_Price']] = df[['Apple_Price', 'Orange_Price', 'Banana_Price']].clip(lower=0)

# Show the head of the price DataFrame and the allocation DataFrame for the last 7 days
print(df.tail(7))
print(allocation_df.tail(7))

         Date  Apple_Price  Orange_Price  Banana_Price
93 2024-04-03     5.631383      5.649344      5.683323
94 2024-04-04     5.688772      5.702507      5.724974
95 2024-04-05     5.756871      5.755801      5.790008
96 2024-04-06     5.808338      5.815842      5.858520
97 2024-04-07     5.876050      5.884249      5.910175
98 2024-04-08     5.921468      5.941060      5.951476
99 2024-04-09     5.960110      5.999712      5.998472
   Apple_Alloc Orange_Alloc Banana_Alloc
93    0.007821     0.531003     0.461176
94    0.025028     0.534477     0.440495
95    0.035462     0.521779      0.44276
96    0.019682     0.522545     0.457773
97    0.031759     0.530251      0.43799
98    0.029418     0.544006     0.426576
99    0.011153     0.560679     0.428169


In [5]:
print(df.head(7))

        Date  Apple_Price  Orange_Price  Banana_Price
0 2024-01-01     1.000000      1.000000      1.000000
1 2024-01-02     1.059320      1.044703      1.060973
2 2024-01-03     1.113319      1.090291      1.107298
3 2024-01-04     1.165488      1.153110      1.152350
4 2024-01-05     1.215365      1.193460      1.199160
5 2024-01-06     1.273206      1.236952      1.247734
6 2024-01-07     1.316633      1.287185      1.289841


In [ ]:
import yfinance as yf

ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2020-12-31")
print(data)

In [ ]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='YOUR_API_KEY', output_format='pandas')
data, meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
print(data)

In [ ]:
import quandl

quandl.ApiConfig.api_key = "YOUR_API_KEY"
data = quandl.get("WIKI/AAPL", start_date="2020-01-01", end_date="2020-12-31")
print(data)

In [ ]:
import requests

url = "https://cloud.iexapis.com/stable/stock/aapl/chart/1y?token=YOUR_API_TOKEN"
response = requests.get(url)
data = response.json()
print(data)